In [ ]:
import numpy as np
import pandas as pd
import pandas as pd
from tqdm import tqdm
import seaborn as sns
from scipy import stats
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib.dates as mdates
import pickle
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, confusion_matrix
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree,export_text, export_graphviz
import graphviz

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
import statsmodels.formula.api as smf
import statsmodels.stats.sandwich_covariance as sw
import joblib
import shap

import math

import scipy.stats as st
# import mord
from scipy import stats 
from scipy.stats import t
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
# from tabulate import tabulate
from pandas.api.types import CategoricalDtype
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression 
from statsmodels.formula.api import ols


import statsmodels.formula.api as smf
import statsmodels.stats.sandwich_covariance as sw
import os
import pickle
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import StratifiedKFold

In [ ]:
df_all = pd.read_csv('forXgboost_addiction.csv',encoding='utf-8')

In [ ]:
df_concated  = pd.read_csv('./data/df_concated.csv',encoding='utf-8')

In [ ]:
gender_dict = {'M': 0, 'F': 1,0:0,1:1}
# gender_dict = {'M': 1, 'F': 2}
gender_str_dict = {'男': 'M', '女':'F' }
sex_dict = {2: 1, 1: 0}


age_segment_dict = {"0-11": 1, "12-17": 2, "18-23": 3, "24-30": 4, "31-40": 5, "41-49": 6, "50+": 7}
fre_city_level_dict = {
    '新一线城市': 1,
    '一线城市': 2,
    '二线城市': 3,
    '三线城市': 4,
    '四线城市': 5,
    '五线城市': 6
}

df_all['gender'] = df_all['gender'].map(gender_dict).astype(float)
df_all['sex_str'] = df_all['sex_str'].map(gender_str_dict)
df_all['sex'] = df_all['sex'].map(sex_dict)

df_all['age_segment'] = df_all['age_segment'].map(age_segment_dict).astype(float)



df_all['fre_city_level'] = df_all['fre_city_level'].map(fre_city_level_dict).astype(float)


# Define the age_segment bins and corresponding labels
bins = [0, 11, 17, 23, 30, 40, 49, np.inf]
labels = [1, 2, 3, 4, 5, 6, 7]

# Fill missing 'age_segment' using pd.cut(), but only for rows where 'age_segment' is NaN
df_all['age_segment'] = np.where(
    df_all['age_segment'].isna(), 
    pd.cut(df_all['age'], bins=bins, labels=labels), 
    df_all['age_segment']
)

# Display the first few rows of the DataFrame to verify the changes
pd.set_option('display.max_columns', None)
df_all.head(2)


In [ ]:
df_all[['addiction_1', 'addiction_2', 'addiction_3', 'addiction_4', 'addiction_5', 'addiction_6',
        'addiction_score', 'soft_criteria', 'hard_criteria', 'age', 'gender', 
        'age_survey', 'sex', 'fre_city_level', 
        'mod_price', 'database', 'submit_time']].dtypes

In [ ]:
df_all['user_id'] = df_all['user_id'].astype(str)
print(df_all['user_id'].nunique())
df_all = df_all.drop_duplicates(subset='user_id', keep='first')
print(df_all['user_id'].nunique())


In [ ]:
df_all.shape

In [ ]:
df_all.groupby('soft_criteria')['user_id'].nunique()


In [ ]:
df_all.groupby('hard_criteria')['user_id'].nunique()


In [ ]:
# regression
p_values = {}

y = 'hard_criteria'
for item in [
    "age",
    "age_survey",

    'C(age_segment)',
    'C(gender)',
    'C(fre_city_level)',
    # 'mod_price',
    
    'hour_0_wt',
 'hour_1_wt',
 'hour_2_wt',
 'hour_3_wt',
 'hour_4_wt',
 'hour_5_wt',
 'hour_6_wt',
 'hour_7_wt',
 'hour_8_wt',
 'hour_9_wt',
 'hour_10_wt',
 'hour_11_wt',
 'hour_12_wt',
 'hour_13_wt',
 'hour_14_wt',
 'hour_15_wt',
 'hour_16_wt',
 'hour_17_wt',
 'hour_18_wt',
 'hour_19_wt',
 'hour_20_wt',
 'hour_21_wt',
 'hour_22_wt',
 'hour_23_wt',
 'morning_wt',
 'noon_wt',
 'afternoon_wt',
 'evening_wt',
 'midnight_wt',
    'watch_all_per_day', 
    "watch_all_per_week",
    "watch_all_per_2weeks", 
    'watch_all_per_session',
    'coverage_per_week',
    'coverage_per_2weeks',
    "category_count_unique",
    "root_category_count_unique",

    'session_all_per_day',
    'pid_SHUA_all_per_day',
    'pid_watch_all_per_day',
    # 'session_all_per_2weeks',
    # 'pid_SHUA_all_per_2weeks',
    # 'pid_watch_all_per_2weeks',
    'duration_per_day',
    'longvideo_count_per_day',
    'shortvideo_count_per_day',

    #  'pastSpent_self',
    # 'futureSpent_self',
]:
    
    logit_mod = smf.logit(formula=f'{y} ~ {item}', data=df_all)
    logit_res = logit_mod.fit()
    
    # Print the results summary
    print("Model =", f'{y} ~ {item}', '\n', logit_res.summary())




    p_values[item] = logit_res.pvalues[1]  # Extract p-value for the independent variable


    # Sort p-values and take the top 10 lowest
    sorted_p_values = dict(sorted(p_values.items(), key=lambda x: x[1])[:])

# Plot the top 10 lowest p-values
plt.figure(figsize=(12, 6))
plt.bar(sorted_p_values.keys(), sorted_p_values.values())
plt.axhline(y=0.05, color='r', linestyle='--')
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.ylabel("p-value", fontsize=14)
plt.title("Top 10 Lowest p-values from Logistic Regression", fontsize=16)
plt.tight_layout()
plt.show()
print(sorted_p_values.keys())

In [ ]:
df_all[[
    "age",
    'gender',
    'watch_all_per_day', 
    'coverage_per_2weeks',
    "category_count_unique",
    'session_all_per_day',
    'midnight_wt',
]]

In [ ]:
## Model for addicted vs non-addicted
SoftOrHard_criteria = 'soft_criteria'
print(f'**** {SoftOrHard_criteria} ****')

# Define target and feature columns
target_column = SoftOrHard_criteria
feature_column = [
    "age", 'gender', 'watch_all_per_day', 'coverage_per_2weeks',
    "category_count_unique", 
    'session_all_per_day', 'midnight_wt',
    'morning_wt', 
    'noon_wt', 
    'afternoon_wt', 
    'evening_wt',
    # scrolling+ watch unique videos vs watch >=1 second(s) unique videos
    'pid_SHUA_all_per_day', 
    'pid_watch_all_per_day',
]

# Define XGBoost parameters
param = {
    'max_depth': 6, 'min_child_weight': 1, 'gamma': 0, 'eta': 0.1,
    'colsample_bytree': 0.7, 'subsample': 0.7, 'eval_metric': 'auc',
    'objective': 'binary:logistic', 'alpha': 0.3, 'lambda': 2.0, 'seed': 42
}
num_round = 110

# Display class distribution
print("\nClass Distribution:")
print(df_all[SoftOrHard_criteria].value_counts())

# Initialize dictionaries to store metrics across folds
metrics_all_train_folds = {
    'train_accuracy': [], 'train_precision': [], 'train_recall': [], 'train_f1_score': [], 'train_roc_auc': []
}

metrics_all_folds = {
    'accuracy': [], 'precision': [], 'recall': [], 'f1_score': [], 'roc_auc': []
}
feature_importances = {feature: 0 for feature in feature_column}

# Set up k-Fold Cross-Validation
smote = SMOTE(random_state=42)
k = 5  # Number of folds
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
fold_index = 1

# Perform k-Fold Cross-Validation
for train_index, test_index in skf.split(df_all, df_all[SoftOrHard_criteria]):
    print(f"\n--- Fold {fold_index}/{k} ---")
    df_train = df_all.iloc[train_index]
    df_test = df_all.iloc[test_index]

    # Prepare data for XGBoost
    xg_train = xgb.DMatrix(df_train[feature_column], label=df_train[SoftOrHard_criteria])
    xg_test = xgb.DMatrix(df_test[feature_column], label=df_test[SoftOrHard_criteria])

    # Calculate scale_pos_weight for imbalance
    pos_count = sum(df_train[SoftOrHard_criteria] == 1)
    neg_count = sum(df_train[SoftOrHard_criteria] == 0)
    param['scale_pos_weight'] = neg_count / pos_count

    # Train the XGBoost model
    bst = xgb.train(param, xg_train, num_round)

     # --- Training Set Evaluation ---
    preds_train = bst.predict(xg_train)
    y_pred_train = (preds_train >= 0.5).astype(int)
    y_train = df_train[SoftOrHard_criteria]

    train_acc = accuracy_score(y_train, y_pred_train)
    train_prec = precision_score(y_train, y_pred_train)
    train_recall = recall_score(y_train, y_pred_train)
    train_f1 = f1_score(y_train, y_pred_train)
    train_roc_auc = roc_auc_score(y_train, preds_train)

    # Store train metrics
    metrics_all_train_folds['train_accuracy'].append(train_acc)
    metrics_all_train_folds['train_precision'].append(train_prec)
    metrics_all_train_folds['train_recall'].append(train_recall)
    metrics_all_train_folds['train_f1_score'].append(train_f1)
    metrics_all_train_folds['train_roc_auc'].append(train_roc_auc)

    print(f"Train Accuracy: {train_acc:.4f}")
    print(f"Train Precision: {train_prec:.4f}")
    print(f"Train Recall: {train_recall:.4f}")
    print(f"Train F1 Score: {train_f1:.4f}")
    print(f"Train ROC AUC: {train_roc_auc:.4f}")


    # --- Test Set Evaluation ---
    preds_test = bst.predict(xg_test)
    y_pred_test = (preds_test >= 0.5).astype(int)
    y_test = df_test[SoftOrHard_criteria]


    # Evaluate metrics for this fold
    test_acc = accuracy_score(y_test, y_pred_test)
    test_prec = precision_score(y_test, y_pred_test)
    test_recall = recall_score(y_test, y_pred_test)
    test_f1 = f1_score(y_test, y_pred_test)
    test_roc_auc = roc_auc_score(y_test, preds_test)

    metrics_all_folds['accuracy'].append(test_acc)
    metrics_all_folds['precision'].append(test_prec)
    metrics_all_folds['recall'].append(test_recall)
    metrics_all_folds['f1_score'].append(test_f1)
    metrics_all_folds['roc_auc'].append(test_roc_auc)

    print(f"Test Accuracy: {test_acc:.4f}")
    print(f"Test Precision: {test_prec:.4f}")
    print(f"Test Recall: {test_recall:.4f}")
    print(f"Test F1 Score: {test_f1:.4f}")
    print(f"Test ROC AUC: {test_roc_auc:.4f}")

    # Save as Pickle
    pkl_path = os.path.join(f'./addict1_xgboost/{SoftOrHard_criteria}_xgboost_model_{fold_index}.pkl')
    with open(pkl_path, 'wb') as model_file:
        pickle.dump(bst, model_file)

    # Accumulate feature importance
    importance_scores = bst.get_score(importance_type='weight')
    for feature, importance in importance_scores.items():
        feature_importances[feature] += importance

    fold_index += 1

# Average feature importances across folds
avg_importances = {feature: importance / k for feature, importance in feature_importances.items()}
sorted_avg_importances = sorted(avg_importances.items(), key=lambda x: x[1], reverse=True)

# Compute summary metrics
metrics_summary = {}
metrics_std = {}
for metric_name, metric_values in metrics_all_folds.items():
    metrics_summary[metric_name] = {
        'best': max(metric_values),
        'average': np.mean(metric_values),
        'lowest': min(metric_values),
    }
    metrics_std[metric_name] = np.std(metric_values)

# Display metrics summary
print("\n=== Metrics Summary over k-Fold Cross-Validation ===")
for metric_name, summary in metrics_summary.items():
    print(f"{metric_name.capitalize()}: Best = {summary['best']:.4f}, Average = {summary['average']:.4f}, Std = {metrics_std[metric_name]:.4f}")

# Plot metrics
metrics = list(metrics_summary.keys())
best_values = [metrics_summary[m]['best'] for m in metrics]
average_values = [metrics_summary[m]['average'] for m in metrics]
lowest_values = [metrics_summary[m]['lowest'] for m in metrics]
std_values = [metrics_std[m] for m in metrics]



x = np.arange(len(metrics))  # the label locations
width = 0.25  # the width of the bars

fig, ax = plt.subplots(figsize=(12, 6))

best_color = '#AC98E1'
average_color = '#B7D1F1'
lowest_color = '#F9C5B3'

rects1 = ax.bar(x - width, best_values, width, label='Best', color=best_color)
rects2 = ax.bar(x, average_values, width, label='Average', color=average_color)
rects3 = ax.bar(x + width, lowest_values, width, label='Lowest', color=lowest_color)

# # Use colors from the 'tab10' colormap
# colors = sns.color_palette("Set2") 


# Adding labels, title, and custom x-axis tick labels
ax.set_ylabel('Metric Value', fontsize=18)
ax.set_title('XGBoost Performance with k-Fold Cross-Validation', fontsize=20)
ax.set_xticks(x)
ax.set_xticklabels([metric.capitalize() for metric in metrics], fontsize=18)
ax.tick_params(axis='y', labelsize=18)

# Adjusting the legend placement to the upper center outside the plot
ax.legend(fontsize=16, loc='upper center', bbox_to_anchor=(0.5, -0.12), fancybox=True, shadow=True, ncol=3)

# Function to attach labels on the bars with spacing
def autolabel(rects, spacing=5):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate(f'{height:.2f}',
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, spacing),  # Use spacing for vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=14, color='black')

# Attach labels to each bar with spacing
autolabel(rects1, spacing=10)
autolabel(rects2, spacing=10)
autolabel(rects3, spacing=10)

# Set y-axis limits for clarity
plt.ylim(0, 1.05)  # Set slightly above 1 for clearer labels

# Add grid lines only on the y-axis
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Adjust layout for better fit and aesthetics
plt.tight_layout()
plt.show()


# Plot average metrics with standard deviation as error bars
fig, ax = plt.subplots(figsize=(10, 6))

# Plot the average values with error bars showing the standard deviation
bars = ax.bar(metrics, average_values, yerr=std_values, capsize=8, color='lightblue', alpha=0.8, edgecolor='black')

# Add labels to the bars
for bar, avg, std in zip(bars, average_values, std_values):
    height = bar.get_height()
    ax.annotate(f'{avg:.4f} ± {std:.4f}',
                xy=(bar.get_x() + bar.get_width() / 2, height*1.13),
                xytext=(0, 5),  # Offset for label position
                textcoords="offset points",
                ha='center', va='bottom', fontsize=14, color='black')

# Customize the plot
ax.set_title('Addicted vs Non-Addicted: XGBoost Performance with Standard Deviation', fontsize=16)
ax.set_ylabel('Metric Value', fontsize=14)
ax.set_xlabel('Performance Metrics', fontsize=14)
ax.set_ylim(0, 1.05)  # Ensure bars fit in the plot
ax.tick_params(axis='x', labelsize=12)
ax.tick_params(axis='y', labelsize=12)

# Add grid lines for clarity
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Show the plot
plt.tight_layout()
plt.show()


### decision tree #####

df_train_best, df_test_best = train_test_split(
    df_all, 
    test_size=0.2, 
    random_state=42,  # Best tryout index or fixed seed
    stratify=df_all[SoftOrHard_criteria]
)

# Prepare X and y for the Decision Tree
X_best = df_train_best[feature_column]  # Features for training
y_best = df_train_best[SoftOrHard_criteria]  # Target variable for training
X_test_best = df_test_best[feature_column]  # Features for testing
y_test_best = df_test_best[SoftOrHard_criteria]  # Target variable for testing

best_roc_auc = 0
best_accuracy = 0
best_precision = 0
best_recall = 0
best_f1 = 0
best_max_depth = None
best_min_samples_leaf = None
best_dt_model = None

# Perform Decision Tree tryouts
for i in [3, 4, 5, 6, 7, 8, 9, 10]:
    for j in [2, 3, 4, 5, 6]:
        print('max_depth: ', i, '; min_samples_leaf: ', j)
        
        # Initialize and fit the Decision Tree model
        dt_model = DecisionTreeClassifier(max_depth=i, min_samples_leaf=j, random_state=42, class_weight='balanced')
        dt_model.fit(X_best, y_best)

        # Predict on the test set
        X_test_best = df_test_best[feature_column]
        y_test_best = df_test_best[SoftOrHard_criteria]
        y_pred_dt = dt_model.predict(X_test_best)

        # Evaluate the model
        dt_roc_auc = roc_auc_score(y_test_best, y_pred_dt)
        
        if dt_roc_auc > best_roc_auc:
            best_roc_auc = dt_roc_auc
            best_dt_model = dt_model
            best_max_depth = i
            best_min_samples_leaf = j

# Print best model details
print("\n*** Decision Tree Model with Best ROC AUC: ***")
print(f"max_depth: {best_max_depth}")
print(f"min_samples_leaf: {best_min_samples_leaf}")
print(f"Best ROC AUC: {best_roc_auc:.4f}")

# --- **Feature Importance Calculation** ---
# Calculate feature importance for the best Decision Tree model
feature_importances = best_dt_model.feature_importances_
feature_importance_df = pd.DataFrame({
    'Feature': feature_column,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

# Print feature importance
print("\n=== Feature Importances ===")
print(feature_importance_df)

# --- **Feature Importance Visualization** ---
plt.figure(figsize=(7, 5))
plt.barh(feature_importance_df['Feature'][:10][::-1], feature_importance_df['Importance'][:10][::-1], color='skyblue')
plt.title("Top 10 Feature Importances (Decision Tree)")
plt.xlabel("Importance")
plt.ylabel("Features")
plt.tight_layout()
plt.show()

In [ ]:
addiction_columns = ['addiction_1', 'addiction_2', 'addiction_3', 'addiction_4', 'addiction_5', 'addiction_6']
# Define addiction columns

# Condition: At least 5 questions with a score of 3 or more
df_all['at_least_5_questions_3plus'] = (df_all[addiction_columns] >= 3).sum(axis=1) >= 5

# Combine the condition for addiction_score >= 18 and at_least_5_questions_3plus
condition = (df_all['soft_criteria'] == 1) & (df_all['addiction_score'] >= 18) & df_all['at_least_5_questions_3plus']

# Create a new column 'hard_criteria_18_atleast5Q' with 1 for rows fulfilling the condition, otherwise 0
df_all['hard_criteria_18_atleast5Q'] = condition.astype(int)

# Debug: Count unique users in each category of 'hard_criteria_18_atleast5Q'
print("\nUnique users by 'hard_criteria_18_atleast5Q':")
print(df_all.groupby('hard_criteria_18_atleast5Q')['user_id'].nunique())

# Assign 3-label criteria
conditions = [
    (df_all['hard_criteria_18_atleast5Q'] == 1),  # Label 2: Hard addiction
    (df_all['soft_criteria'] == 1) & (df_all['hard_criteria_18_atleast5Q'] == 0),  # Label 1: Soft addiction but not hard addiction
    (df_all['soft_criteria'] == 0)  # Label 0: Non-addicted
]

choices = [2, 1, 0]

# Create the new column '3_label_criteria_18_atleast5Q'
df_all['3_label_criteria_18_atleast5Q'] = np.select(conditions, choices, default=np.nan)

# Debug: Check value counts for the new column
print("\nValue counts for '3_label_criteria_18_atleast5Q':")
print(df_all['3_label_criteria_18_atleast5Q'].value_counts())


df_all_hard_vs_soft = df_all[df_all['soft_criteria'] == 1]

In [ ]:
## severe vs mildly addicted people 
SoftOrHard_criteria = 'hard_criteria'
print(f'**** {SoftOrHard_criteria} ****')
# Define target and feature columns
target_column = SoftOrHard_criteria
feature_column = [
    "age", 'gender', 
    'watch_all_per_day',
    'coverage_per_2weeks',
    "category_count_unique",
    'session_all_per_day', 
    'midnight_wt',
    'morning_wt', 'noon_wt', 'afternoon_wt', 
    'evening_wt',
    # # scrolling+ watch unique videos vs watch >=1 second(s) unique videos
    'pid_SHUA_all_per_day', 'pid_watch_all_per_day',
]

# Define XGBoost parameters
param = {
    'max_depth': 3, 'min_child_weight': 5, 'gamma': 2, 'eta': 0.01,
    'colsample_bytree': 0.7, 'subsample': 0.6, 'eval_metric': 'auc',
    'objective': 'binary:logistic', 'alpha': 0.3, 'lambda': 2.0, 'seed': 42
}
num_round = 130



# Display class distribution
print("\nClass Distribution:")
print(df_all_hard_vs_soft[SoftOrHard_criteria].value_counts())

# Initialize dictionaries to store metrics across folds
metrics_all_train_folds = {
    'train_accuracy': [], 'train_precision': [], 'train_recall': [], 'train_f1_score': [], 'train_roc_auc': []
}

metrics_all_folds = {
    'accuracy': [], 'precision': [], 'recall': [], 'f1_score': [], 'roc_auc': []
}
feature_importances = {feature: 0 for feature in feature_column}

# Set up k-Fold Cross-Validation
smote = SMOTE(random_state=42)
# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
k = 5  # Number of folds
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
fold_index = 1

# Perform k-Fold Cross-Validation
for train_index, test_index in skf.split(df_all_hard_vs_soft, df_all_hard_vs_soft[SoftOrHard_criteria]):
    print(f"\n--- Fold {fold_index}/{k} ---")
    df_train = df_all_hard_vs_soft.iloc[train_index]
    df_test = df_all_hard_vs_soft.iloc[test_index]

    print(f"Train set size: {len(df_train)}")
    print(f"Test set size: {len(df_test)}")

    # Prepare data for XGBoost
    xg_train = xgb.DMatrix(df_train[feature_column], label=df_train[SoftOrHard_criteria])
    xg_test = xgb.DMatrix(df_test[feature_column], label=df_test[SoftOrHard_criteria])

    # Calculate scale_pos_weight for imbalance
    pos_count = sum(df_train[SoftOrHard_criteria] == 1)
    neg_count = sum(df_train[SoftOrHard_criteria] == 0)
    param['scale_pos_weight'] = neg_count / pos_count

    # Train the XGBoost model
    bst = xgb.train(param, xg_train, num_round)

    # --- Training Set Evaluation ---
    preds_train = bst.predict(xg_train)
    y_pred_train = (preds_train >= 0.5).astype(int)
    y_train = df_train[SoftOrHard_criteria]

    train_acc = accuracy_score(y_train, y_pred_train)
    train_prec = precision_score(y_train, y_pred_train)
    train_recall = recall_score(y_train, y_pred_train)
    train_f1 = f1_score(y_train, y_pred_train)
    train_roc_auc = roc_auc_score(y_train, preds_train)

    # Store train metrics
    metrics_all_train_folds['train_accuracy'].append(train_acc)
    metrics_all_train_folds['train_precision'].append(train_prec)
    metrics_all_train_folds['train_recall'].append(train_recall)
    metrics_all_train_folds['train_f1_score'].append(train_f1)
    metrics_all_train_folds['train_roc_auc'].append(train_roc_auc)

    print(f"Train Accuracy: {train_acc:.4f}")
    print(f"Train Precision: {train_prec:.4f}")
    print(f"Train Recall: {train_recall:.4f}")
    print(f"Train F1 Score: {train_f1:.4f}")
    print(f"Train ROC AUC: {train_roc_auc:.4f}")


    # --- Test Set Evaluation ---
    preds_test = bst.predict(xg_test)
    y_pred_test = (preds_test >= 0.5).astype(int)
    y_test = df_test[SoftOrHard_criteria]


    # Evaluate metrics for this fold
    test_acc = accuracy_score(y_test, y_pred_test)
    test_prec = precision_score(y_test, y_pred_test)
    test_recall = recall_score(y_test, y_pred_test)
    test_f1 = f1_score(y_test, y_pred_test)
    test_roc_auc = roc_auc_score(y_test, preds_test)

    metrics_all_folds['accuracy'].append(test_acc)
    metrics_all_folds['precision'].append(test_prec)
    metrics_all_folds['recall'].append(test_recall)
    metrics_all_folds['f1_score'].append(test_f1)
    metrics_all_folds['roc_auc'].append(test_roc_auc)

    print(f"Test Accuracy: {test_acc:.4f}")
    print(f"Test Precision: {test_prec:.4f}")
    print(f"Test Recall: {test_recall:.4f}")
    print(f"Test F1 Score: {test_f1:.4f}")
    print(f"Test ROC AUC: {test_roc_auc:.4f}")

    pkl_path = os.path.join(f'./addict1_xgboost/{SoftOrHard_criteria}_xgboost_model_{fold_index}.pkl')
    with open(pkl_path, 'wb') as model_file:
        pickle.dump(bst, model_file)

    # Accumulate feature importance
    importance_scores = bst.get_score(importance_type='weight')
    for feature, importance in importance_scores.items():
        feature_importances[feature] += importance

    fold_index += 1

# Average feature importances across folds
avg_importances = {feature: importance / k for feature, importance in feature_importances.items()}
sorted_avg_importances = sorted(avg_importances.items(), key=lambda x: x[1], reverse=True)

# Compute summary metrics
metrics_summary = {}
metrics_std = {}
for metric_name, metric_values in metrics_all_folds.items():
    metrics_summary[metric_name] = {
        'best': max(metric_values),
        'average': np.mean(metric_values),
        'lowest': min(metric_values),
    }
    metrics_std[metric_name] = np.std(metric_values)

# Display metrics summary
print("\n=== Metrics Summary over k-Fold Cross-Validation ===")
for metric_name, summary in metrics_summary.items():
    print(f"{metric_name.capitalize()}: Best = {summary['best']:.4f}, Average = {summary['average']:.4f}, Std = {metrics_std[metric_name]:.4f}")

# Plot metrics
metrics = list(metrics_summary.keys())
best_values = [metrics_summary[m]['best'] for m in metrics]
average_values = [metrics_summary[m]['average'] for m in metrics]
lowest_values = [metrics_summary[m]['lowest'] for m in metrics]
std_values = [metrics_std[m] for m in metrics]



x = np.arange(len(metrics))  # the label locations
width = 0.25  # the width of the bars

fig, ax = plt.subplots(figsize=(10, 6))

best_color = '#AC98E1'
average_color = '#B7D1F1'
lowest_color = '#F9C5B3'

rects1 = ax.bar(x - width, best_values, width, label='Best', color=best_color)
rects2 = ax.bar(x, average_values, width, label='Average', color=average_color)
rects3 = ax.bar(x + width, lowest_values, width, label='Lowest', color=lowest_color)
ax.axhline(y=0.5, color='r', linestyle='--', label='Random Guess')



# Adding labels, title, and custom x-axis tick labels
ax.set_ylabel('Metric Value', fontsize=18)
ax.set_title('Severely vs Mildly Addicted Users: XGBoost Performance with k-Fold Cross-Validation', fontsize=20)
ax.set_xticks(x)
ax.set_xticklabels([metric.capitalize() for metric in metrics], fontsize=18)
ax.tick_params(axis='y', labelsize=18)

# Adjusting the legend placement to the upper center outside the plot
ax.legend(fontsize=16, loc='upper center', bbox_to_anchor=(0.5, -0.12), fancybox=True, shadow=True, ncol=3)

# Function to attach labels on the bars with spacing
def autolabel(rects, spacing=5):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate(f'{height:.2f}',
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, spacing),  # Use spacing for vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=14, color='black')

# Attach labels to each bar with spacing
autolabel(rects1, spacing=10)
autolabel(rects2, spacing=10)
autolabel(rects3, spacing=10)

# Set y-axis limits for clarity
plt.ylim(0, 1.05)  # Set slightly above 1 for clearer labels

# Add grid lines only on the y-axis
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Adjust layout for better fit and aesthetics
plt.tight_layout()
plt.show()


# Plot average metrics with standard deviation as error bars
fig, ax = plt.subplots(figsize=(10, 6))

# Plot the average values with error bars showing the standard deviation
bars = ax.bar(metrics, average_values, yerr=std_values, capsize=8, color='lightblue', alpha=0.8, edgecolor='black')

# Add labels to the bars
for bar, avg, std in zip(bars, average_values, std_values):
    height = bar.get_height()
    ax.annotate(f'{avg:.4f} ± {std:.4f}',
                xy=(bar.get_x() + bar.get_width() / 2, height+0.19),
                xytext=(0, 5),  # Offset for label position
                textcoords="offset points",
                ha='center', va='bottom', fontsize=14, color='black')

# Customize the plot
ax.set_title('XGBoost Performance with Standard Deviation', fontsize=16)
ax.set_ylabel('Metric Value', fontsize=14)
ax.set_xlabel('Performance Metrics', fontsize=14)
ax.set_ylim(0, 1.05)  # Ensure bars fit in the plot
ax.tick_params(axis='x', labelsize=12)
ax.tick_params(axis='y', labelsize=12)

# Add grid lines for clarity
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
df_10000 = pd.read_csv('10000_user_data.csv',encoding='utf-8')
df_10000 = df_10000.rename(columns={'age_kuaishou': 'age'})
df_10000['user_id'] = df_10000['user_id'].astype(str)

df_survey = pd.read_csv('df_survey.csv',encoding='utf-8')

In [ ]:
required_columns = df_all.columns

common_columns=[ 'user_id','hour_0_wt',
 'hour_1_wt',
 'hour_2_wt',
 'hour_3_wt',
 'hour_4_wt',
 'hour_5_wt',
 'hour_6_wt',
 'hour_7_wt',
 'hour_8_wt',
 'hour_9_wt',
 'hour_10_wt',
 'hour_11_wt',
 'hour_12_wt',
 'hour_13_wt',
 'hour_14_wt',
 'hour_15_wt',
 'hour_16_wt',
 'hour_17_wt',
 'hour_18_wt',
 'hour_19_wt',
 'hour_20_wt',
 'hour_21_wt',
 'hour_22_wt',
 'hour_23_wt',
 'morning_wt',
 'noon_wt',
 'afternoon_wt',
 'evening_wt',
 'midnight_wt',
 'watch_all_per_day',
 'watch_all_per_week',
 'watch_all_per_2weeks',
 'watch_all_per_session',
 'session_all_per_day',
 'pid_SHUA_all_per_day',
 'pid_watch_all_per_day',
 'duration_per_day',
 'longvideo_count_per_day',
 'shortvideo_count_per_day',
 'coverage_per_week',
 'coverage_per_2weeks',
 'category_count_unique',
 'root_category_count_unique',
 'month']

df_survey = df_survey[list(common_columns)]
df_survey['user_id'] = df_survey['user_id'].astype(str)
df_concated['user_id'] = df_concated['user_id'].astype(str)
df_survey = df_survey.merge(df_concated[['user_id','addiction_score','soft_criteria','hard_criteria','age','gender','age_segment','age_survey','sex','sex_str','fre_city_level','database','submit_time']],on='user_id',how='left')


df_survey['hard_criteria'] = 0

# Merge the corresponding values from df_all into past_119_interaction_all
df_survey = df_survey.merge(
    df_all[['user_id', 'hard_criteria']], 
    on='user_id', 
    how='left', 
    suffixes=('', '_from_df_all')
)


print(df_survey[~df_survey['hard_criteria'].isna()]['user_id'].nunique())

user_ids_to_update = df_survey[(df_survey['watch_all_per_day'] == 0) & 
                                              (df_survey['month'] == 0)]['user_id'].unique()

columns_to_remove = ['hard_criteria','soft_criteria', 'addiction_score', '3_label_criteira']


# Filter both datasets to include only the required columns
df_survey_filt = df_survey[required_columns]

df_10000_filt = df_10000[required_columns]

df_all_forlabeling = pd.concat([df_survey_filt, df_10000_filt], ignore_index=True)
print(df_all_forlabeling.shape)


df_all_forlabeling['pid_0s_per_day_nunique'] = df_all_forlabeling['pid_SHUA_all_per_day'] - df_all_forlabeling['pid_watch_all_per_day']


df_all_forlabeling['gender'] = df_all_forlabeling['gender'].map(gender_dict).astype(float)
gender_str_dict = {'男': 'M', '女':'F' }
df_all_forlabeling['sex_str'] = df_all_forlabeling['sex_str'].map(gender_str_dict)


df_all_forlabeling['fre_city_level'] = df_all_forlabeling['fre_city_level'].map(fre_city_level_dict).astype(float)

# Define the age_segment bins and corresponding labels
bins = [0, 11, 17, 23, 30, 40, 49, np.inf]
labels = [1, 2, 3, 4, 5, 6, 7]

# Fill missing 'age_segment' using pd.cut(), but only for rows where 'age_segment' is NaN
df_all_forlabeling['age_segment'] = np.where(
    df_all_forlabeling['age_segment'].isna(), 
    pd.cut(df_all_forlabeling['age'], bins=bins, labels=labels), 
    df_all_forlabeling['age_segment']
)
df_all_forlabeling['age_survey_segment'] = np.nan
df_all_forlabeling['age_survey_segment'] = np.where(
    df_all_forlabeling['age_survey_segment'].isna(), 
    pd.cut(df_all_forlabeling['age_survey'], bins=bins, labels=labels), 
    df_all_forlabeling['age_survey_segment']
)

df_all_forlabeling = df_all_forlabeling[df_all_forlabeling['month'] <= 9]

df_all_forlabeling['month_chronological_order'] = 10 - df_all_forlabeling['month']

df_all_forlabeling.head()

In [ ]:
with open('./addict1_xgboost/soft_criteria_xgboost_model_best.pkl', 'rb') as model_file:
    best_model = pickle.load(model_file)


feature_column = [
    "age", 'gender', 'watch_all_per_day', 'coverage_per_2weeks',
    "category_count_unique", 
    'session_all_per_day', 'midnight_wt',
    'morning_wt', 
    'noon_wt', 
    'afternoon_wt', 
    'evening_wt',
    # # scrolling+ watch unique videos vs watch >=1 second(s) unique videos
    'pid_SHUA_all_per_day', 
    'pid_watch_all_per_day',
]

# Check that all required feature columns are present in the dataset
missing_features = [feature for feature in feature_column if feature not in df_all_forlabeling.columns]
if missing_features:

    print(f"need missing feature columns: {missing_features}")

# Prepare the dataset for prediction
d_matrix = xgb.DMatrix(df_all_forlabeling[feature_column])  # Only include required feature columns

# Predict the target labels using the loaded model
predicted_labels = best_model.predict(d_matrix)

# Convert probabilities to binary labels (0 or 1)
df_all_forlabeling['preds_soft'] = (predicted_labels >= 0.5).astype(int)


with open('./addict1_xgboost/hard_criteria_xgboost_model_severely_mildly_best.pkl', 'rb') as model_file:
    best_model = pickle.load(model_file)

# Check that all required feature columns are present in the dataset
missing_features = [feature for feature in feature_column if feature not in df_all_forlabeling.columns]
if missing_features:
    print(f"Missing feature columns detected: {missing_features}")
    # Optionally create the missing feature columns with default values (e.g., 0 or NaN)
    for feature in missing_features:
        df_all_forlabeling[feature] = 0  # Adjust default value if necessary

# Step 1: Filter dataset to include only users where preds_soft == 1
soft_criteria_subset = df_all_forlabeling[df_all_forlabeling['preds_soft'] == 1]

# Step 2: Prepare the DMatrix for XGBoost prediction using the filtered subset
d_matrix = xgb.DMatrix(soft_criteria_subset[feature_column])  # Only include required feature columns

# Step 3: Predict the target labels using the loaded model
predicted_probs = best_model.predict(d_matrix)

# Step 4: Convert probabilities to binary labels (0 or 1) for preds_hard
soft_criteria_subset['preds_hard'] = (predicted_probs >= 0.5).astype(int)

# Step 5: Merge predictions back into the original dataset
# Keep only the 'user_id' and 'preds_hard' columns from the filtered dataset
soft_criteria_subset = soft_criteria_subset[['user_id','month', 'preds_hard']]

# Merge the predicted 'preds_hard' back into the full dataset
df_all_forlabeling = df_all_forlabeling.merge(soft_criteria_subset, on=['user_id','month'], how='left', suffixes=('', '_temp'))

# Step 6: Ensure all users have a 'preds_hard' value
# Fill NaN values in 'preds_hard' with 0
df_all_forlabeling['preds_hard'].fillna(0, inplace=True)

# Step 7: Drop the temporary column (if created during merging)
if 'preds_hard_temp' in df_all_forlabeling.columns:
    df_all_forlabeling.drop(columns=['preds_hard_temp'], inplace=True)

# make sure the first month data is updated into it
month_0_data = df_all_forlabeling[(df_all_forlabeling['month'] == 0) & (~df_all_forlabeling['soft_criteria'].isna())]
df_all_forlabeling.loc[month_0_data.index, 'preds_soft'] = month_0_data['soft_criteria']
df_all_forlabeling.loc[month_0_data.index, 'preds_hard'] = month_0_data['hard_criteria']

In [ ]:
df_all_forlabeling.to_csv('dataset.csv', encoding='utf-8',index=False)